In [31]:
! pip install selenium
! pip install html_table_parser

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for beautifulsoup4==4.4.1 from https://files.pythonhosted.org/packages/cf/69/9abfdab06490af5e0233bcebe3f617ec128486d94ea987ad4f77b9332eef/beautifulsoup4-4.4.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/81.4 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.4 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 81.4/81.4 kB 1.5 MB/s eta 0:00:00
  Created wheel for html_table_parser: filename=html_table_parser-0.1.0-py3-none-any.whl size=4638 sha256=2f599eac9f28c620288fd47ba00512a4bafaa8e8464d32c88e500e5168094cc8
  Stored in directory: c:\users\melid\appdata\local\pip\cache\wheels\3f\61\3e\205c3503f7c244357c1ba105368ba70095a71a075624689850
Successfully built html_table_parser
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.12.2
    Uninstalling beau

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [3]:
driver = webdriver.Chrome()

In [4]:
url = "https://statiz.sporki.com/stats/?m=main&m2=all"
driver.get(url)

In [5]:
driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/section/div[1]/div/button[4]').click()

# 팀 옵션 드롭다운 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/button').click()
# 삼성 라이온즈 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/ul/li[6]').click()

# 수비 > 내야 선택
driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/section/div[6]/ul/li[3]/a').click()

In [6]:
from urllib.request import urlopen

import collections
collections.Callable = collections.abc.Callable

from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import pandas as pd

In [7]:
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

In [8]:
temp = soup.find_all('table')
p = parser.make2d(temp[0])    
for i in range(len(p)):
    print(i, p[i])
print(len(p))

0 ['Rank', 'Name', 'Team', 'Sort▼', 'G', 'GS', 'IP', 'ifH', 'FC', 'PO', 'Ass', 'E', 'BBO', 'BBO%', 'F%', 'RF9', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도']
1 ['Rank', 'Name', 'Team', '수비 관련 득점 기여도-종합 RAA', 'G', 'GS', 'IP', 'ifH', 'FC', 'PO', 'Ass', 'E', 'BBO', 'BBO%', 'F%', 'RF9', 'Range RAA', 'Err RAA', 'DP RAA', 'Bt RAA', '종합 RAA▼', '/144']
2 ['1', '맥키넌', '24  1B', '5.12', '25', '25', '218.0', '8', '0', '160', '18', '1', '72', '87.5', '99.4', '7.35', '5.57', '0.04', '', '-0.48', '5.12', '20.50']
3 ['2', '이재현', '24  SS', '4.12', '22', '21', '182.0', '5', '0', '37', '42', '2', '32', '78.1', '97.5', '3.91', '6.08', '-0.35', '-1.61', '', '4.12', '24.72']
4 ['3', '강한울', '24  3B', '3.65', '9', '3', '41.0', '3', '1', '1', '14', '0', '12', '66.7', '100.0', '3.29', '3.12', '0.15', '0.38', '', '3.65', '29.18']
5 ['4', '김영웅', '24  SS', '3.23', '19', '19', '171.2', '13', '0', '40', '51', '3', '71', '77.5', '96.8', '4.77', '1.16', '0.11', '1.96'

In [9]:
num = (len(p) // 10)-1

start_num = 2
for i in range(num*2+1, 1, -1):
    n = start_num * 10 + (i)
    p.pop(n)
    if i % 2 == 0:
        start_num -= 1

In [10]:
data=pd.DataFrame(p)
data=data[2:]
data.columns = ["Rank", "Name", "Position","Total RAA2","G", "GS","IP", "ifH","FC","PO","Ass","E","BBO","BBO%","F%","RF9","Range RAA", " Err RAA", "DP RAA","Bt RAA","Total RAA", "/144"]
data.drop(columns=['Total RAA2'], inplace=True)
data['Position'].replace({"24  ":""}, regex=True, inplace=True)
data['Team']="Samsung"
print(data)

   Rank Name Position   G  GS     IP ifH FC   PO Ass  ...   BBO%     F%  \
2     1  맥키넌       1B  25  25  218.0   8  0  160  18  ...   87.5   99.4   
3     2  이재현       SS  22  21  182.0   5  0   37  42  ...   78.1   97.5   
4     3  강한울       3B   9   3   41.0   3  1    1  14  ...   66.7  100.0   
5     4  김영웅       SS  19  19  171.2  13  0   40  51  ...   77.5   96.8   
6     5  김재상       2B  22  19  151.1   4  0   32  44  ...   87.2   98.7   
7     6  김호진       3B   9   5   43.2   2  0    3   9  ...   77.8  100.0   
8     7  안주형       3B   2   0    3.0   0  0    1   2  ...  100.0  100.0   
9     8  김재혁       1B   4   4   31.0   0  0   21   4  ...  100.0  100.0   
10    9  류지혁       2B  17  13  110.1   4  0   14  21  ...   72.2   97.2   
11   10  공민규       1B   2   0    5.0   1  0    5   2  ...   85.7  100.0   
12   11  김지찬       2B   6   0    9.0   0  0    0   0  ...                 
13   12  김재상       3B   1   0    1.0   0  0    0   0  ...                 
14   13  강한울       1B   1

In [11]:
data.to_csv("Samsung_field_IF.csv")